In [113]:
import time

from pylsl import StreamInlet, resolve_stream
from brainflow.data_filter import DataFilter, WindowOperations, WaveletTypes, AggOperations, WindowOperations, WaveletDenoisingTypes, WaveletExtensionTypes, NoiseEstimationLevelTypes, ThresholdTypes

import numpy as np
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pygame.mixer
from datetime import datetime

In [2]:
pygame.mixer.init()

# Helper Functions

In [119]:
def stream_data(duration):
    '''
    Input: duration (in seconds) of the EEG recording.

    Output: A numpy array of size or shape (channel, timesteps). Where channel is 4 for Muse2
        and timestep varies for each recording.
    '''


    # PyLSL to collect the EEG data streamed from BlueMuse.
    # Code adapted from: https://github.com/chkothe/pylsl/blob/master/examples/ReceiveData.py
    streams = resolve_stream('type', 'EEG')
    inlet = StreamInlet(streams[0])
    data = []

    # This captures data for a specified duration (in seconds).
    start_time = time.time()
    while time.time() - start_time < duration:    
        sample, timestamp = inlet.pull_sample()    
        data.append(sample)
    data_array = np.array(data).T
    data_array = np.ascontiguousarray(data_array)
    return data_array
    

In [3]:
def pad_or_trim(data, target_length):
    '''
    Input: data (coming from EEG recording) with shape (channels, timesteps).
            target_length to either pad or trim the timesteps to the desired level.
        
    Output: A modified version of data which the timesteps trimmed.

    For example, recording with Muse2 can yield 250 - 265 timesteps due to certain inconsistencies.
    We can use this function to always trim/pad the data recorded and limit it to 256 timesteps (From (4, 260) to (4, 256))

    '''
    if data.shape[1] < target_length:
        pad_width = target_length - data.shape[1]
        data = np.pad(data, ((0, 0), (0, pad_width)), 'constant', constant_values=0)
    else:
        data = data[:, :target_length]
    return data

## Muse2 Board

### Board Initialisation

This section has been removed as Brainflow had connectivity issues. Streaming is now done through BlueMuse application and PyLSL library.

### Parameters for Data Collection

In [131]:
# This is so we don't overwrite the entire data when we re-run the collection.
session_features = None
session_labels = None
save = False

In [120]:
done_sfx = pygame.mixer.Sound('Sound Effects/Enter.mp3')
sampling_freq = 256 # Muse2's sampling frequency. (How many timesteps per second)

duration = 2 # How long should each sample be? (in seconds)
rest_time = 2 # How many seconds in between each recording?

In [121]:
iterations = 1
midi_note = 1 # Initially, let's just try 1 2 3 for do re mi (think of the word along with the sound)

### Data Collection

In [132]:
print("Be sure to only blink during the rest times allocated.")

session_start = datetime.now().strftime('%d-%m-%Y %H_%M')

for i in range(iterations):
    print("Iteration ", i + 1, "/", iterations)
    print("Think of the note: ", midi_note, "in ", rest_time, " second(s).")
    time.sleep(rest_time)

    ## Streaming ##
    #data = stream_data(duration=duration)
    data = data_array.T.copy()
    if done_sfx:
        done_sfx.play()

    ## End Stream ## 
    ## Start Data Storage ## 
    
    # This pads/trims the data so it's 256 timesteps long (or whatever the sampling frequency is)
    data = pad_or_trim(data, sampling_freq*duration) # -> (channels, sampling_freq) shape. Or (4, 256) for Muse2. [EEG only]

    data = np.expand_dims(data, 0)
    label = np.array([midi_note]) # Saves the current note as a feature.
    if session_features is None:
        session_features = data 
        session_labels = label
    else:
        session_features = np.append(session_features, data, axis=0)
        session_labels = np.append(session_labels, label, axis=0)

    
    # This clears the Python notebook output.
    clear_output()

### Data Processing

In [ ]:
session_features_denoised = session_features.copy()
session_features_decomposed = None

In [ ]:
# We would have to denoise the data.
### TO IMPLEMENT: Data denoising.
# Refer to this: https://brainflow.readthedocs.io/en/stable/Examples.html#python-denoising
# Note: This function performs denoising IN PLACE.
nfft = DataFilter.get_nearest_power_of_two(sampling_freq) # This is used to extract the band power.

for idx in range(len(session_features_denoised)):
    for channel in range(len(session_features_denoised[idx])):
        # This part performs the denoising INPLACE.
        DataFilter.perform_wavelet_denoising(session_features_denoised[idx][channel],
                                     WaveletTypes.BIOR3_9,
                                     3,
                                     WaveletDenoisingTypes.SURESHRINK,
                                     ThresholdTypes.HARD,
                                     WaveletExtensionTypes.SYMMETRIC,
                                     NoiseEstimationLevelTypes.FIRST_LEVEL
                                     )
        
for idx in range(len(session_features_denoised)):
    for channel in range(len(session_features_denoised[idx])):
        # Next, we'd also have to extract the different wave states from each channel.
        # Refer to this: https://brainflow.readthedocs.io/en/stable/Examples.html#python-band-power

        psd = DataFilter.get_psd_welch(session_features_denoised[idx][channel], 128, 128 // 2, sampling_freq,
                                WindowOperations.BLACKMAN_HARRIS)

        band_power_delta =  DataFilter.get_band_power(psd, 0.5, 4.0)
        band_power_theta =  DataFilter.get_band_power(psd, 4.1, 8.0)
        band_power_alpha = DataFilter.get_band_power(psd, 8.1, 12.0)
        band_power_beta = DataFilter.get_band_power(psd, 12.1, 30.0)
        band_power_gamma = DataFilter.get_band_power(psd, 14.1, 30.0)

        band_power = np.array([band_power_delta, band_power_theta, band_power_alpha, band_power_beta, band_power_gamma])
        band_power = np.expand_dims(band_power, axis=0)

        if session_features_decomposed is None:
            session_features_decomposed = band_power
        else:
            session_features_decomposed = np.append(session_features_decomposed, band_power, axis=0)
        


# However, through this, we still have the same amount of features. Therefore, we have to extract relevant features
# from each of the wave.

### Data Saving

In [ ]:
# Save the data as a .npy file
if save:
    np.save(f'{session_start} RAW.npy', session_features)
    np.save(f'{session_start} DECOMPOSED.npy', session_features_decomposed)
    np.save(f'{session_start} DENOISED.npy', session_features_denoised)